In [1]:
import numpy as np

from grg_pheno_sim.multi_grg_phenotype import sim_phenotypes_multi_grg
from grg_pheno_sim.model import grg_causal_mutation_model


The following commands only serve the purpose of converting the VCF zip file into GRGs that will be used for phenotype simulation.

In [2]:
%%script bash --out /dev/null
if [ ! -f test-200-samples.grg ]; then
  grg construct -p 10 -t 2 ../data/test-200-samples.vcf.gz --out-file test-200-samples.grg
fi

In [3]:
%%script bash --out /dev/null
if [ ! -f test-200-samples_copy.grg ]; then
  grg construct -p 10 -t 2 ../data/test-200-samples.vcf.gz --out-file test-200-samples_copy.grg
fi

In [4]:
grg_list = ["test-200-samples.grg", "test-200-samples_copy.grg"]
#this is the list of GRG files to be loaded in 

We will first demonstrate loading all GRG files into RAM and simulating phenotypes. Causal mutations are sampled from each GRG, and the genetic values are obtained for the samples. The combined genetic dataframe is the addition of each GRG's genetic values (for each causal mutation ID). Noise is sampled at the end and added to obtain the phenotypes.

NOTE: It is necessary for each GRG to have the same number of samples.

In [5]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_phenotype = True #check for normalizing phenotyopes

normalize_genetic_values_before_noise = False

heritability = [0.33, 0.25]

load_all_into_RAM = True #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, normalize_phenotype=normalize_phenotype, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability)


Loaded test-200-samples.grg into RAM
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0       1.542758                   0
1                0      -0.615642                   1
2                1      -3.334669                   0
3                1      -8.701385                   1
4                2       5.724925                   0
..             ...            ...                 ...
395            197      -5.473324                   1
396            198       7.793645                   0
397            198       2.128440                   1
398            199      14.981245                   0
399            199       1.083515                   1

[400 rows x 3 columns]
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      24.977116                   0
1                0     -

In [6]:
multi_grg_multi_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-0.529293,-0.220295,-0.749588
1,1,0,-0.220450,-0.647147,-0.867597
2,0,1,-0.881266,0.713365,-0.167901
3,1,1,-0.814167,-1.571987,-2.386154
4,0,2,-0.744129,0.487034,-0.257095
...,...,...,...,...,...
395,1,197,-0.218334,-0.063209,-0.281544
396,0,198,0.196661,-0.612732,-0.416071
397,1,198,0.047755,0.548440,0.596196
398,0,199,-0.024600,-1.974953,-1.999552


We now perform similar simulations, but by loading the GRGs into RAM sequentially (instead of all together).

In [7]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_genetic_values_before_noise = False

heritability = [0.33, 0.25]

load_all_into_RAM = False #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_seq_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability)


Loaded test-200-samples.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      -1.793143                   0
1                0      26.684182                   1
2                1      -9.208692                   0
3                1       8.148092                   1
4                2      -5.902249                   0
..             ...            ...                 ...
395            197      30.528290                   1
396            198      -2.076871                   0
397            198      38.922745                   1
398            199      -2.671229                   0
399            199      15.135616                   1

[400 rows x 3 columns]
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      -9.729285                   0
1                0     -

In [8]:
multi_grg_multi_seq_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-11.522428,12.366081,0.843654
1,1,0,14.130809,-10.542896,3.587913
2,0,1,-23.501074,34.260971,10.759897
3,1,1,-5.250385,19.052086,13.801701
4,0,2,-34.135463,5.342505,-28.792959
...,...,...,...,...,...
395,1,197,13.921939,6.323837,20.245776
396,0,198,-19.110866,-9.519748,-28.630614
397,1,198,26.365942,-0.453364,25.912578
398,0,199,-7.447051,-9.714752,-17.161803
